In [2]:
import pandas as pd

df = pd.read_csv("arabic_sentiment_dataset.csv")
df.head()


,تعليق,التصنيف
0,هذا المنتج رائع جداً وأنصح به!,إيجابي
1,أسوأ تجربة شراء قمت بها.,سلبي
2,الخدمة جيدة نوعاً ما.,محايد
3,لم يعجبني أبداً ولن أشتريه مرة أخرى.,سلبي
4,منتج ممتاز وسعر مناسب.,إيجابي


In [3]:
!pip install transformers arabert nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 9.2 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186456 sha256=119d2a0f816222066883b701203b4b7c0a597ff8143ed41bc18411e001468081
  Stored in directory: /root/.cache/pip/wheels/94/08/b4/78657b1541bb704b088317b52429ee4016d9888fe47dbb130f
Successfully built emoji


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# تحميل نموذج AraBERT المصنف للمشاعر
model_name = "aubmindlab/bert-base-arabertv02-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
texts = df['تعليق'].tolist()

# مثال على ترميز أول تعليق
encoding = tokenizer(texts[0], truncation=True, padding=True, return_tensors="pt")
print(encoding)


{'input_ids': tensor([[    2,   434,  8222, 13149,     1,   958,  1124,   680,     8,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

# تحميل النموذج و Tokenizer مرة تانية إذا لازم
model_name = "aubmindlab/bert-base-arabertv02-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# ضع النموذج بوضع التقييم (غير تدريبي)
model.eval()

# لائحة لوضع النتائج
results = []

# أسماء التصنيفات (نفرض: 0 سلبي، 1 محايد، 2 إيجابي)
labels = ['سلبي', 'محايد', 'إيجابي']

# تحليل كل تعليق في الملف
for text in df['تعليق']:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        pred_class = torch.argmax(probs, dim=1).item()
        results.append(labels[pred_class])

# إضافة النتائج إلى الجدول الأصلي
df['تحليل_المشاعر'] = results
df.head()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,تعليق,التصنيف,تحليل_المشاعر
0,هذا المنتج رائع جداً وأنصح به!,إيجابي,سلبي
1,أسوأ تجربة شراء قمت بها.,سلبي,محايد
2,الخدمة جيدة نوعاً ما.,محايد,إيجابي
3,لم يعجبني أبداً ولن أشتريه مرة أخرى.,سلبي,محايد
4,منتج ممتاز وسعر مناسب.,إيجابي,محايد


In [8]:
df.to_csv("نتائج_تحليل_المشاعر.csv", index=False)


# 🧠 تحليل المشاعر باستخدام TextBlob

## 💡 فكرة المشروع:
تحليل المشاعر لتعليقات باللغة العربية باستخدام أدوات بسيطة على Google Colab.

## 🧰 الأدوات المستخدمة:
- Python
- Pandas
- TextBlob (مع دعم اللغة العربية)
- Google Colab

## ⚙️ خطوات العمل:
1. تحميل البيانات (CSV)
2. تنظيف النصوص إذا لزم الأمر
3. تحليل المشاعر باستخدام TextBlob
4. تصدير النتائج

## 📊 نتائج مبدئية:
النموذج استطاع تحديد الاتجاه العام للمشاعر (سلبي، إيجابي، محايد) بدقة مقبولة كمشروع مبدئي بسيط.

---

> ✅ المشروع ما زال أوليًّا ويمكن تطويره لاحقاً باستخدام نماذج أكثر تطورًا مثل AraBERT.


with open("README.md", "w", encoding="utf-8") as f:
    f.write("""\
# تحليل المشاعر من تعليقات باللغة العربية باستخدام AraBERT

🎯 **وصف المشروع:**
هذا المشروع يهدف إلى تحليل المشاعر في تعليقات مكتوبة باللغة العربية، وتحديد ما إذا كانت التعليقات إيجابية أو سلبية، باستخدام نموذج التعلم العميق AraBERT.

📦 **المكتبات والأدوات المستخدمة:**
- Python
- Google Colab
- Pandas, Numpy
- Transformers (HuggingFace)
- AraBERT model (aubmindlab/bert-base-arabertv02)
- Scikit-learn

📁 **محتوى المشروع:**
- `sentiment_analysis_project.ipynb`: كود المشروع الكامل للتدريب والتنبؤ.
- `arabic_sentiment_dataset.csv`: بيانات التدريب (تعليقات + مشاعرها).
- `arabic_sentiment_results.csv`: النتائج بعد التنبؤ.

📊 **خطوات العمل:**
1. تحميل البيانات.
2. تنظيف ومعالجة النصوص.
3. تحميل نموذج AraBERT.
4. تحويل النصوص إلى تمثيلات رقمية باستخدام tokenizer.
5. تدريب النموذج على البيانات.
6. اختبار النموذج وتوليد النتائج.
7. تحليل الأداء بدقة و Recall و F1 Score.

🚀 **النتائج:**
النموذج أظهر دقة جيدة في تصنيف المشاعر خاصة ضمن البيانات المتوازنة.

🧠 **أسباب اختيار AraBERT:**
- لأنه من أقوى نماذج اللغة العربية.
- مدرب على بيانات عربية منوعة (أخبار، تويتر، الخ...).
- مناسب تماماً لمهام تحليل النصوص والتصنيف.

📌 **ملاحظات إضافية:**
- المشروع تم تنفيذه بالكامل على Google Colab للاستفادة من موارد الحوسبة السحابية.
- لم يتم استخدام جهاز محلي لأن المواصفات محدودة.
- المشروع قابل للتطوير لاحقاً باستخدام بيانات حقيقية من منصات اجتماعية.

📫 **للتواصل / الدعم:**
يمكن مراسلتي عبر GitHub لأي استفسار أو اقتراح.

"ibrahim.qreet97@gmail.com"")
